## 1. 概述
- Android 四大组件所涉及的多进程间的通信底层都是依赖于 Binder IPC 机制。例如当进程 A 中的 Activity 要向进程 B 中的 Service 通信，这便需要依赖于 Binder IPC。不仅于此，整个 Android 系统架构中，大量采用了 Binder 机制作为 IPC（进程间通信）方案，当然也存在部分其他的 IPC 式，比如 Zygote 通信便是采用 socket。

---

## 2. Binder
### 2.1 IPC 原理
- 每个 Android 的进程，只能运行在自己进程所拥有的虚拟地址空间
- 对应一个 4GB 的虚拟地址空间，其中 3GB 是用户空间，1GB 是内核空间，内核空间的大小可通过参数配置调整。对于用户空间，不同进程之间彼此是不能共享的，而内核空间却是可共享的。
- Client 进程向 Server 进程通信，恰恰是利用进程间可共享的内核内存空间来完成底层通信工作的，Client 端与 Server 端进程往往采用 ioctl 等方法跟内核空间的驱动进行交互。

![image](binder0_page1.png)

### 2.2 Binder 原理
- Binder 通信采用 C/S 架构，从组件视角来说，包含 Client、Server、ServiceManager 以及 binder 驱动，其中 ServiceManager 用于管理系统中的各种服务。

![image](binder0_page2.png)

- 图解
  - 无论是注册服务和获取服务的过程都需要 ServiceManager，需要注意的是此处的 Service Manager 是指 Native 层的 ServiceManager（C++），并非指 framework 层的 ServiceManager(Java)。
  - ServiceManager 是整个 Binder 通信机制的大管家，是 Android 进程间通信机制 Binder 的守护进程。当 Service Manager 启动之后，Client 端和 Server 端通信时都需要先获取 Service Manager 接口，才能开始通信服务。
  - 1. 注册服务(addService)：Server 进程要先注册 Service 到 ServiceManager。该过程：Server 是客户端，ServiceManager 是服务端。
  - 2. 获取服务(getService)：Client 进程使用某个 Service 前，须先向 ServiceManager 中获取相应的 Service。该过程：Client 是客户端，ServiceManager 是服务端。
  - 3. 使用服务：Client 根据得到的 Service 信息建立与 Service 所在的 Server 进程通信的通路，然后就可以直接与 Service 交互。该过程：client 是客户端，server 是服务端。
  
---

### 2.3 C/S 模式
- BpBinder(客户端) 和 BBinder(服务端) 都是 Android 中 Binder 通信相关的代表，它们都从 IBinder 类中派生而来

![image](binder0_page3.png)

- client端：BpBinder.transact() 来发送事务请求
- server端：BBinder.onTransact() 会接收到相应事务

## 3. 提纲
![image](binder0_page4.png)

### 第一部分
- 首先阅读 Binder系列5 — 注册服务(addService) 和 Binder系列6 — 获取服务(getService)，这两个过程都需要于 ServiceManager 打交道
- 这两个过程在开始之前都需要 Binder系列4 — 获取 Service Manager
- 要获取 Service Manager，就需要 Binder系列3 — 启动 Service Manager
- 最终调用到底层 Binder 底层驱动，这时需要了解 Binder系列1 — Binder Driver 初探和 Binder系列2 — Binder Driver 再探

### 第二部分
- 接下来，可以看看上层 Binder 系列7 — framework 层分析的 Binder 架构情况
- 系统工程师：Binder系列8 — 如何使用 Binder
- 应用开发工程师：Binder系列9 — 如何使用 AIDL
- 最后是总结：Binder系列10 — 总结

---

## 4. 源码目录
```
/framework/base/core/java/               (Java)
/framework/base/core/jni/                (JNI)
/framework/native/libs/binder            (Native)
/framework/native/cmds/servicemanager/   (Native)
/kernel/drivers/staging/android          (Driver)
```

### 4.1 Java framework
```
/framework/base/core/java/android/os/  
    - IInterface.java
    - IBinder.java
    - Parcel.java
    - IServiceManager.java
    - ServiceManager.java
    - ServiceManagerNative.java
    - Binder.java  

/framework/base/core/jni/    
    - android_os_Parcel.cpp
    - AndroidRuntime.cpp
    - android_util_Binder.cpp (核心类)
```

### 4.2 Native framework
```
/framework/native/libs/binder         
    - IServiceManager.cpp
    - BpBinder.cpp
    - Binder.cpp
    - IPCThreadState.cpp (核心类)
    - ProcessState.cpp  (核心类)

/framework/native/include/binder/
    - IServiceManager.h
    - IInterface.h

/framework/native/cmds/servicemanager/
    - service_manager.c
    - binder.c
```

### 4.3 Kernel
```
/kernel/drivers/staging/android/
    - binder.c
    - uapi/binder.h
```

---